In [2]:
import pandas as pd

#### Load Data

In [44]:
path = "/Users/marinawiemers/Downloads/gbg_escooter_2022_voi.csv"
# df = pd.read_csv(path)
df = pd.read_csv(path, delimiter=';')

#### Uniformise Data for Scripts

In [46]:
# Some data sources use varying names, scripts are based on this
df = df.rename(columns={'distance':'escooter_distance', 'transit_Time':'transit_time', 'transit_total_duration':'transit_totaltime', 'transit_walkTime':'transit_walkdistance', 'car_duration':'car_time', 'carDistance':'car_distance', 'walk_duration':'walk_time', 'walkDistance':'walk_distance', 'walkTime_y':'walk_time', 'walkDistance_y':'walk_distance'})

cols = ['id', 'o_time', 'd_time', 'o_lat', 'o_lng', 'd_lat', 'd_lng',
       'escooter_distance', 'escooter_time', 'transit_time',
       'transit_totaltime', 'transit_walkdistance', 'transit_transitdistance',
       'car_time', 'car_distance', 'walk_time', 'walk_distance', 'type',
       'ratio_', 'U_walk', 'U_bike', 'U_PT', 'U_car', 'U_taxi', 's', 'P_walk',
       'P_bike', 'P_PT', 'P_car', 'P_taxi', 'GHG']

cols_final = list(set(cols) & set(df.columns))
df = df[cols_final]

In [47]:
df['o_time'] = pd.to_datetime(df['o_time'], errors='coerce')
df['d_time'] = pd.to_datetime(df['d_time'], errors='coerce')
df = df.sort_values(by=["o_time"]).reset_index(drop=True)
df['day'] = df['o_time'].dt.day_name()
df['month'] = df['o_time'].dt.month_name()
df['Month'] = df['o_time'].dt.month
df['year'] = df['o_time'].dt.year
df['Date'] = df['o_time'].dt.day
df['hour'] = df['o_time'].dt.hour

In [48]:
# Initial data source used "vio", scripts are based on this
df['type'] = df['type'].replace('voi', 'vio') if 'type' in df.columns else 'vio'

In [49]:
df['reduced_time'] = ((df['P_walk'] * df['walk_time']) + (df['P_bike'] * df['escooter_time']) + (df['P_PT'] * df['transit_totaltime']) + (df['P_car'] * df['car_time']) + (df['P_taxi'] * df['car_time'])) - df['escooter_time']
df['transit_walktime'] = df['transit_totaltime'] - df['transit_time']

#### Filter for specific time- and area range

<mark> Adjust timeframe and area (coordinates) accordingly</mark>

In [57]:
# Filter for timeframe
df_range = df[(df['year']==2022) & (((df['Month']==8)) | ((df['Month']==9)))]
df_range=df_range.reset_index(drop=True)
print('#Datapoints - total: ' + str(df_range.shape[0]))

# Filter for area
# e.g. from prev datasets; 
# STH - 59.39, 59.28, 18.18, 17.94
# GBG - 57.74, 57.67, 12.02, 11.90
north, south, east, west = 57.74, 57.67, 12.02, 11.90
mask_lng = (df_range['o_lng'] > east) | (df_range['d_lng'] > east) | (df_range['o_lng'] < west) | (df_range['d_lng'] < west)
mask_lat = (df_range['o_lat'] > north) | (df_range['d_lat'] > north) | (df_range['o_lat'] < south) | (df_range['d_lat'] < south)
mask = mask_lng | mask_lat
df_city = df_range[~mask].reset_index(drop=True)
print('#Datapoints -  area filter: ' + str(df_city.shape[0]))

#Datapoints - total: 101751
#Datapoints -  area filter: 98592


#### Filter for outliers based on E-Scooter specs

* Max distance based on battery capacity
* Filter out round trips
* Speed specifications

In [58]:
# Filter for distance
df_city = df_city[df_city['escooter_distance'] < 20000]
df_city = df_city[df_city['escooter_distance'] > 50] 
print('#Datapoints - distance filter: ' + str(df_city.shape[0]))
      
# Filter for trip duration and speed
df_city = df_city[(df_city['escooter_time'] > 30) & (df_city['escooter_time'] < 3600)]
df_city['speed']= df_city['escooter_distance']/df_city['escooter_time']
df_city = df_city[(df_city['speed'] < 10) & (df_city['speed'] > (2/3.6))]
print('#Datapoints - speed & time filter: ' + str(df_city.shape[0]))

#Datapoints - distance filter: 98592
#Datapoints - speed & time filter: 96477


#### Save to file

In [ ]:
df_city.to_csv('city_escooter_year_months_provider.csv', index=False)